## Apache kafka - strumieniowanie danych

1. Sprawdź listę topiców

Pamiętaj aby przejść do katalogu domowego.

```bash
cd ~ 
kafka/bin/kafka-topics.sh --list --bootstrap-server broker:9092
```

2. Utwórz nowy topic o nazwie `mytopic`

```bash
kafka/bin/kafka-topics.sh --create --topic mytopic  --bootstrap-server broker:9092
```

3. Utwórz producenta 

Ten skrypt pozwoli Ci wprowadzać eventy ręcznie przez terminal.
Opcje `--property` są dodatkowe i wprowadzone tylko do analizy przeprowadzanej w tym przykładzie.

```bash
kafka/bin/kafka-console-producer.sh --bootstrap-server broker:9092 --topic mytopic --property "parse.key=true" --property "key.separator=:"
```

4. Otwórz nowy terminal w miejscu gdzie znajduje się plik test_key_value.py i uruchom program Consumenta na Sparku

Pamiętaj, iż Apache Spark nie posiada domyślnego konektora do Kafki dlatego uruchom proces poprzez komendę spark-submit i pobierz odpowiedni pakiet w scali.

```bash
spark-submit --packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0  test_key_value.py
```

5. W terminalu z uruchominionym producentem wpisz teskt w postaci: 
```bash
jan:45
alicja:20
```
i sprawdz co pojawia się w oknie aplikacji consumenta.

Po zakończeniu pokazu ctrl+c zamyka zarówno okno producenta jak i aplikacje sparkową

## REST API z wykorzystaniem biblioteki Flask 

Zanim przejdziemy do utworzenia podstawowej aplikacji Flask przyda się zrozumienie jak działa dekorator.

Zdefiniujmy następującą funkcję: 

In [1]:
def test():
    print("test z funkcji")

Dodajmy kolejną funkcję która pozwoli nam uruchomić nasz test.

In [2]:
def dekorator(func):
    def inner():
        print("funkcja wewnętrzna")
        func()
    return inner()

Tak przygotowane funkcje możemy wspólnie uruchomić jako:

In [3]:
dekorator(test)

funkcja wewnętrzna
test z funkcji


bardziej naturalnie możemy uruchomić dowolną funkcję już podczas jej deklaracji:

In [4]:
@dekorator
def test2():
    print("drukuj test2")

funkcja wewnętrzna
drukuj test2


### Aplikacja Flask

Poniźszy domyślny kod wygeneruje serwer z trzema podstronami. 
> Aplikacji nie możemy uruchomić w notatniku bezpośrednio - uruchomi się i zablokuje pozostałe komórki notatnika. Dlatego zapiszmy ją do pliku



In [5]:
%%file app1.py
from flask import Flask

app = Flask(__name__)

@app.route('/hello')
def say_hello():
    return "Hello World"

@app.route('/')
def say_he():
    return "Hello z innej strony"

@app.route('/test')
def say():
    return "test"

if __name__ == '__main__':
    app.run()

Writing app1.py


Możemy wykorzystać biblioteki do uruchomienia kodu jako podproces. Dodatkowo uruchomimy bibliotekę odpowiadającą za komunikację. 

In [6]:
import subprocess
import requests

In [9]:
p = subprocess.Popen(["python", "app1.py"])

 * Serving Flask app 'app1'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


Odpytajmy stronę uruchomionego serwisu

In [10]:
adres_url = "http://127.0.0.1:5000/hello"
response = requests.get(adres_url)

127.0.0.1 - - [28/Mar/2025 02:18:27] "GET /hello HTTP/1.1" 200 -


In [11]:
print(response.content)

b'Hello World'


In [12]:
adres_url = "http://127.0.0.1:5000/helloooo"
response = requests.get(adres_url)

127.0.0.1 - - [28/Mar/2025 02:19:40] "GET /helloooo HTTP/1.1" 404 -


In [13]:
print(response.content)

b'<!doctype html>\n<html lang=en>\n<title>404 Not Found</title>\n<h1>Not Found</h1>\n<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>\n'


In [14]:
p.kill()

In [15]:
%%file app2.py

from flask import Flask
from flask import request

app = Flask(__name__)

@app.route('/hello', methods=['GET'])
def say_hello():
    name = request.args.get("name", "") # tutaj leci str
    title = request.args.get("title", "")
    if name:
        resp = f"Hello {title} {name}" if title else f"Hello {name}"
    else:
        resp = f"Hello {title}" if title else "Hello"
    return resp

if __name__ == '__main__':
    app.run(port=5005)

Writing app2.py


In [16]:
p = subprocess.Popen(["python", "app2.py"])

 * Serving Flask app 'app2'
 * Debug mode: off


 * Running on http://127.0.0.1:5005
Press CTRL+C to quit


In [17]:
response = requests.get("http://127.0.0.1:5005/hello")
if response.status_code == 200: 
    print("dziala",response.content)
else: 
    print("Nie dziala",response.content)

dziala b'Hello'


127.0.0.1 - - [28/Mar/2025 02:21:59] "GET /hello HTTP/1.1" 200 -


In [20]:
response = requests.get("http://127.0.0.1:5005/hello?name=Sebastian&title=dr")
response.content

127.0.0.1 - - [28/Mar/2025 02:22:26] "GET /hello?name=Sebastian&title=dr HTTP/1.1" 200 -


b'Hello dr Sebastian'